Metaclasse
- Cria classes de forma dinâmica
- Pode ser útil na criação de frameworks, onde as classes precisam ter comportamento específico.

In [64]:
class MeuMeta(type):
    def __new__(cls,nome,bases,dct):
        dct['novo_atributo'] = 'Valor adicionado pela MetaClasse'
        return super().__new__(cls,nome,bases,dct)


In [65]:
class MinhaClasse(metaclass=MeuMeta):
    pass

In [66]:
obj = MinhaClasse()
obj.novo_atributo # type: ignore

'Valor adicionado pela MetaClasse'

In [67]:
class ValidadorMeta(type):
    def __new__(cls,nome,bases,dct):
        validacoes = dct.get('validacoes', {})

        for attr, tipo in validacoes.items():
            if not callable(tipo):
                raise TypeError(f'O tipo de validação para {attr} deve ser uma função.')
            
            # Adiciona função de validação
            def valida_func(self, value, attr=attr, tipo=tipo):
                if not isinstance(value, tipo):
                    raise ValueError(f'{attr} deve ser do tipo {tipo.__name__}')
                
                self.__dict__[attr] = value
            
            # Renomeia a função para evitar problemas de escopo
            valida_func.__name__ = f"set_{attr}"
            dct[f"set_{attr}"] = valida_func

        return super().__new__(cls,nome,bases,dct)

In [68]:
class Usuario(metaclass=ValidadorMeta):
    validacoes = {
        'nome': str,
        'idade': int,
    }

    #MÉTODO CONSTRUTOR
    def __init__(self, nome, idade): 
        self.set_nome(nome)
        self.set_idade(idade)




In [75]:
try:
    user = Usuario("Alice", 30)
    print(f"Nome: {user.nome}, Idade: {user.idade}")

    # Tentativa de atribuir um valor inválido
    user.set_idade("Trinta")#idade atribuída no tipo errado propositalmente para fins didáticos
    #user.set_nome(30)

except ValueError as e:
    print(e)

Nome: Alice, Idade: 30
idade deve ser do tipo int
